In [ ]:
import pandas as pd
dataset = pd.read_csv("../../Data/cleaned_v1.csv")
dataset

In [ ]:
from gensim.models import Word2Vec
w2v = Word2Vec.load("../../FeatureEngineering/Vectorize/w2v_300_5window_cleaned_24krow_skipgram.model")

In [18]:
from keras_preprocessing.text import Tokenizer
from keras_preprocessing import sequence
from keras_preprocessing.sequence import pad_sequences

import numpy as np

tokenizer = Tokenizer(filters='!"#$%&()*+,-./:;<=>?@[\\]^`{|}~\t\n')
tokenizer.fit_on_texts(["Toi di hoc   abs boi cao"])
# sequences = tokenizer.texts_to_sequences(dataset['Content_cleaned'])
sequences = tokenizer.texts_to_sequences(["Toi di hoc abs boi cao"])

# maxlen = max(dataset["Length_content_cleaned"])
X = sequence.pad_sequences(sequences, maxlen=10)
# print('X:\n', X[:10])


In [19]:
X

array([[0, 0, 0, 0, 1, 2, 3, 4, 5, 6]], dtype=int32)

In [ ]:
y = pd.get_dummies(dataset['Label'])
y

In [ ]:
import numpy as np

embedding_dim = 300
word_index = tokenizer.word_index
num_words = len(word_index) + 1
embedding_matrix = np.zeros((num_words, embedding_dim))
print('num_words:', num_words)
for word, i in word_index.items():
    if word in w2v.wv.key_to_index:
            embedding_matrix[i] = w2v.wv[word]

In [ ]:

from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense, LSTM, Bidirectional, SimpleRNN, Dropout

model = Sequential()
model.add(Embedding(num_words, embedding_dim, input_length=maxlen,weights=[embedding_matrix],trainable=False))
model.add(SimpleRNN(units=300))
model.add(Dense(units=128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(units=32, activation='relu'))
model.add(Dense(6, activation='softmax'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, y , test_size= 0.3, random_state= 42)
X_train.shape, Y_train.shape, X_test.shape, Y_test.shape

In [ ]:
from keras.callbacks import EarlyStopping
callback = EarlyStopping(monitor = "val_accuracy", patience= 4)

In [ ]:
model.fit(X_train, Y_train, validation_split=0.1, epochs= 100, callbacks=[callback])

In [ ]:
from matplotlib import pyplot as plt
plt.plot(model.history.history['accuracy'])
plt.plot(model.history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()
from matplotlib import pyplot as plt
plt.plot(model.history.history['loss'])
plt.plot(model.history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
loss, accuracy = model.evaluate(X_test, Y_test)
print('Test Loss:', loss)
print('Test Accuracy:', accuracy)

In [ ]:
from sklearn.metrics import classification_report
# Example classification report data
report = classification_report(y_pred, Y_test)
report